In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import torch

# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from models.unet.unet import UNet
from models.unet.unet_formes import UNetFormes
from src.data_processing.dataset_loader import CoastData
from src.models.data_management.data_loader import DataLoaderManager

import cv2

In [ ]:
# Load the data to split it and save it to a dict
path = os.path.abspath(os.path.join(os.getcwd(), "../../data/patchify_4_classes/"))

In [ ]:
unet = UNet(num_classes=4, experiment_name="test_experiments", use_mlflow=False)

In [ ]:
# Load the data to the model
data = unet.load_data(path, UNetFormes)

Not classified percentage: 42.260%

Land percentage: 31.173%

Sea percentage: 26.415%

Shoreline percentage: 0.153%

---

not_classified = 42,260 / 42,260 =  1

land = 42,260 / 31,173 = 1,36

sea = 142,260 / 26,415 = 1,60

shoreline = 42,260 / 0,153 = 276,14


In [ ]:
weight = [1, 1.36, 1.60, 276.14]

artifact_path = os.path.abspath(os.path.join(path, "../../artifacts/"))

# Train the model
# unet.train(epochs=50, loss_function_name="BCEWithLogitsLoss", weight=weight, artifact_path=artifact_path)

# Predictions

In [ ]:
# Predict one patch
artifact_path = os.path.abspath(os.path.join(path, "../../artifacts/"))
data_path = os.path.abspath(os.path.join(os.getcwd(), "../../data/patchify_4_classes/"))

data = DataLoaderManager.load_data(data_path)
index = 0 # 72
img_path = data["test"]['images'][index]
mask_path = data["test"]['masks'][index]

unet.load_model(os.path.abspath(os.path.join(artifact_path, "2025-02-19-15-28-50/models/best_model.pth")))

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

# pred = unet.predict(img)
pred = unet.predict(img_path)

print(np.unique(mask, return_counts=True))
print(np.unique(pred, return_counts=True))


fig, ax = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle(f"Img path: {img_path.split('/')[-1]}", y=0.05)

ax[0].imshow(img)
ax[0].set_title("Image")

ax[1].imshow(mask)
ax[1].set_title("Mask")

ax[2].imshow(pred.squeeze())
ax[2].set_title("Prediction")

plt.show()

In [ ]:
# Predict the full image
artifact_path = os.path.abspath(os.path.join(path, "../../artifacts/"))
data_path = os.path.abspath(os.path.join(os.getcwd(), "../../data/processed_4_classes/"))

coast_data = CoastData(data_path)
split = coast_data.split_data()

data = DataLoaderManager.load_data(split)
index = 100 # 25 # 72
img_path = data["test"]['images'][index]
mask_path = data["test"]['masks'][index]

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

unet.load_model(os.path.abspath(os.path.join(artifact_path, "2025-02-19-15-28-50/models/best_model.pth")))

pred = unet.predict_patch(img_path, combination="max")

print(np.unique(mask, return_counts=True))
print(np.unique(pred, return_counts=True))

fig, ax = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle(f"Img path: {img_path.split('/')[-1]}", y=0.05)

ax[0].imshow(img)
ax[0].set_title("Image")

ax[1].imshow(mask)
ax[1].set_title("Mask")

ax[2].imshow(pred.squeeze())
ax[2].set_title("Prediction")

plt.show()

# Metrics

In [ ]:
from src.models.metrics import Metrics

artifact_path = os.path.abspath(os.path.join(os.getcwd(), "../../artifacts/"))

num_classes = 4
metrics = {
    "train": Metrics(phase="train", num_classes=num_classes, average=None, loss=False),
    "validation": Metrics(phase="val", num_classes=num_classes, average=None, loss=False),
    "test": Metrics(phase="test", num_classes=num_classes, average=None, loss=False)
}

# Load the data to split it and save it to a dict
unet = UNet(num_classes=num_classes, experiment_name="test_experiments", use_mlflow=False)
unet.load_model(os.path.abspath(os.path.join(artifact_path, "2025-02-19-15-28-50/models/best_model.pth")))

data_path = os.path.abspath(os.path.join(os.getcwd(), "../../data/processed_4_classes/"))

coast_data = CoastData(data_path)
split = coast_data.split_data()

data = DataLoaderManager.load_data(split)

for split in data:
    print(f"Split: {split}")
    for img_path, mask_path in zip(data[split]["images"], data[split]["masks"]):
        pred = unet.predict_patch(img_path, combination="max")
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = torch.from_numpy(mask).float() 
        metrics[split].update(mask, pred)
        break

    metrics[split].compute(epoch=0)
    print(metrics[split].get_last_epoch_info())